# Practica 2

**Nombre:** Jorge Osvaldo Gonzalez Gonzalez  
**e-mail:** jorge.gonzalez@alumnos.udg.mx

### MODULES

In [1]:
import math

import os 

import numpy as np
import pandas as pd

import plotly.graph_objects as go

from scipy.stats import wrapcauchy
from scipy.stats import levy_stable

from scipy.spatial import distance

### CLASES

In [2]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

### FUNCTIONS

#### Provided

In [3]:
###############################################################################################
# Turning angle
###############################################################################################
def turning_angle(vec_a, vec_b, vec_c):
    """
    Arguments:
        vec_a: First detection coordinates
        vec_b: Second detection coordinates
        vec_c: Third detection coordinates
    Returns:
        theta: Turning angle 
    """
    ab = vec_b-vec_a
    norm_ab = np.linalg.norm(ab)
    
    bc = vec_c-vec_b
    norm_bc = np.linalg.norm(bc)

    dot_p = np.dot(ab, bc)
    
    cross_p = np.cross(ab, bc)
    orient = np.sign(cross_p)
    if orient == 0:
        orient = 1
    
    cos_theta = dot_p/(norm_ab*norm_bc+np.finfo(float).eps)
    theta = np.arccos(np.around(cos_theta,4)) * orient
    return theta



#### Own

##### Parsers


In [4]:
def np_arrays_to_df(arrays, column_names): 
    df_res = pd.DataFrame(columns=column_names)
    for i in range(0, len(arrays[0])):
        df_res = pd.concat([df_res, pd.DataFrame(columns=column_names, data=[[arrays[j][i]  for j in range(0, len(column_names))]])], ignore_index=True)
    return df_res

##### Files

In [5]:
def save_df_to_csv(df_in, path, name):
    os.makedirs(path, exist_ok=True)
    df_in.to_csv(f'{path}/{name}.csv', index=False)

##### Correlated Random Walk Generation

In [6]:
def generate_crw(steps, speed, coef):
    cur_pos = Vec2d(speed, 0)
    crw_df = pd.DataFrame([{'x': 0, 'y': 0}])
    crw_turns = wrapcauchy.rvs(c=coef, loc=0, size=steps-1)
    for i in range(1, steps):
        cur_pos = cur_pos.rotated(crw_turns[i-1])
        crw_df = pd.concat([crw_df, pd.DataFrame([{'x': crw_df.iloc[i-1, 0] + cur_pos.x, 'y': crw_df.iloc[i-1, 1] + cur_pos.y}])]) 
    return crw_df

##### Levy Flight Generation

In [7]:
def generate_levy(steps, speed, cauchy, alpha, beta, std_steps):
    cur_pos_lw = Vec2d(speed, 0)
    lw_2d_df = pd.DataFrame([{"x": 0, "y": 0}])
    lw_displacements = levy_stable.rvs(alpha=alpha, beta=beta, loc=std_steps)
    # Generate trajectory by accumulation of displacements4
    i = 1
    while i <= steps:
        turn_angle = wrapcauchy.rvs(c=cauchy, loc=0)
        cur_pos_lw = cur_pos_lw.rotated(turn_angle)
        cur_steps = abs(round(levy_stable.rvs(alpha=alpha, beta=beta, loc=std_steps))) 

        for j in range(0, int(cur_steps)):
            lw_2d_df = pd.concat([lw_2d_df, pd.DataFrame([{'x': lw_2d_df.x[i-1]+cur_pos_lw.x, 'y': lw_2d_df.y[i-1]+cur_pos_lw.y}])], ignore_index=True)
            i+=1
            if i > steps:
                break
    return lw_2d_df

##### MSD

In [8]:
###################################################################################
# MSD VECTOR
###################################################################################
def MSDVector(df_in):
    displacement_vector = np.zeros(df_in.shape[0] - 1)
    for tau in range(1,df_in.shape[0]):
        ## start - Add your code here
        MSD = np.array([distance.euclidean(df_in.iloc[i-tau], df_in.iloc[i]) ** 2
            for i in range(tau, df_in.shape[0], 1)])
        displacement_vector[tau-1] = MSD.mean()
    return displacement_vector

## Actividad 1: Path Length - (BM1 vs BM2 vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Calcular métrica utilizando exclusivamente funciones de NumPy
* Guardar métricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [9]:
# Load existing trajectories
# BM speed = 3
BM_2d_df_3 = pd.read_csv('trajectories/brownian_3.csv')

# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
CRW9_2d_df_6 = pd.read_csv('trajectories/crw_6_9.csv') 

In [10]:
# Compute path length
## start - Add your code here
dis_bm2_s3 = pd.DataFrame([{"dis": 0}])
dis_bm2_s6 = pd.DataFrame([{"dis": 0}])
dis_crw9_s6 = pd.DataFrame([{"dis": 0}])

for i in range(1, BM_2d_df_3.shape[0]):
    pointA = np.array((BM_2d_df_3.iloc[i-1].x_pos, BM_2d_df_3.iloc[i-1].y_pos)) 
    pointB = np.array((BM_2d_df_3.iloc[i].x_pos, BM_2d_df_3.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_bm2_s3 = pd.concat([dis_bm2_s3, pd.DataFrame([{"dis": dis_bm2_s3.iloc[i-1].dis + dis}])], ignore_index=True)

for i in range(1, BM_2d_df_6.shape[0]):
    pointA = np.array((BM_2d_df_6.iloc[i-1].x_pos, BM_2d_df_6.iloc[i-1].y_pos)) 
    pointB = np.array((BM_2d_df_6.iloc[i].x_pos, BM_2d_df_6.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_bm2_s6 = pd.concat([dis_bm2_s6, pd.DataFrame([{"dis": dis_bm2_s6.iloc[i-1].dis + dis}])], ignore_index=True)

for i in range(1, CRW9_2d_df_6.shape[0]):
    pointA = np.array((CRW9_2d_df_6.iloc[i-1].x_pos, CRW9_2d_df_6.iloc[i-1].y_pos)) 
    pointB = np.array((CRW9_2d_df_6.iloc[i].x_pos, CRW9_2d_df_6.iloc[i].y_pos))
    dis = round(np.sqrt(np.sum(np.square(pointA - pointB))), 3)
    dis_crw9_s6 = pd.concat([dis_crw9_s6, pd.DataFrame([{"dis": dis_crw9_s6.iloc[i-1].dis + dis}])], ignore_index=True)
## end - Add your code here


In [11]:
# Plotting
# Init figure
fig_path_length = go.Figure()

# First trace BM1
## start - Add your code here
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_bm2_s3.dis,
            line = dict(width=2),
            mode = 'lines',
            name = 'BM speed:3',
            showlegend = True
        )
    )
## end - Add your code here


# Second trace BM2
## start - Add e your code here
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_bm2_s6.dis,
            line = dict(width=4, color='rgb(35,35,35)'),
            mode = 'lines',
            name = 'BM speed:6',
            showlegend = True
        )
    )
## end - Add your code here


# Third trace CRW
fig_path_length.add_trace(
        go.Scatter(
            x = np.arange(len(dis_bm2_s3)),
            y = dis_crw9_s6.dis,
            line = dict(width=2, dash='dash', color='magenta'),
            mode = 'lines',
            name = 'CRW speed:6 c:0.9',
            showlegend = True
        )
    )
## start - Add your code here
    
## end - Add your code here

fig_path_length.update_layout(title_text='Random Walks - Path Length',
                          autosize=False,
                          width=800,
                          height=400
                          )
fig_path_length.show()


## Actividad 2: Mean Squared Displacement - (Brownian vs CRW)

* Cargar trayectorias en **Pandas** Data Frame
* Guardar metricas en **Pandas** Data Frame
* Visualizar con **plotly**

In [12]:
# Load existing trajectories
# BM speed = 6
BM_2d_df_6 = pd.read_csv('trajectories/brownian_6.csv')

# Load existing trajectories
# CRW speed = 6, c = 0.9
CRW_2d_df_9 = pd.read_csv('trajectories/crw_6_9.csv')

In [13]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter3d(x = BM_2d_df_6.x_pos,
                 y = BM_2d_df_6.y_pos,
                 z = BM_2d_df_6.index,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'BM 2d',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter3d(x = CRW_2d_df_9.x_pos,
                 y = CRW_2d_df_9.y_pos,
                 z = CRW_2d_df_9.index,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'CRW 2d',
                 showlegend = True))

fig_3d.show()

In [14]:
# Generating MSD Vectors from own implementation
MSD_BM = MSDVector(BM_2d_df_6)
MSD_CRW = MSDVector(CRW_2d_df_9)

# Generating MSD DataFrame from own implementation
MSD_BM6_CRW6_dis_df = np_arrays_to_df(arrays=[MSD_BM, MSD_CRW], column_names=["MSD_BM", "MSD_CRW"])

# Writing to csv from own implementation
save_df_to_csv(MSD_BM6_CRW6_dis_df, './metrics', 'met_msd_bm_crw')

In [15]:
# Init figure
fig_msd = go.Figure()

# first trace MSD_BM
## start - Add your code here
fig_msd.add_trace(
        go.Scatter(
            x = np.arange(MSD_BM6_CRW6_dis_df.shape[0]),
            y = MSD_BM6_CRW6_dis_df.MSD_BM,
            line = dict(width=2, color='rgb(35,35,35)'),
            mode = 'lines',
            name = 'BM speed:6',
            showlegend = True
        )
    )
## end - Add your code here


# Second trace MSD_CRW
## start - Add your code here
fig_msd.add_trace(
        go.Scatter(
            x = np.arange(MSD_BM6_CRW6_dis_df.shape[0]),
            y = MSD_BM6_CRW6_dis_df.MSD_CRW,
            line = dict(width=2, dash='dash', color='magenta'),
            mode = 'lines',
            name = 'CRW speed:6',
            showlegend = True
        )
    )
## end - Add your code here


fig_msd.show()

## Actividad 3: Turning-angle Distribution - (Dist. origen vs Dist. observada)

* Generar CRWs con dos exponentes diferentes
* Guardar trayectorias en Pandas Data Frame
* Obtener Turning-angle distribution
* Guardar metricas en **Pandas** Data Frame
* Comparar en gráfica distribución origen vs distribución observada (Histograma)
* Visualizar con **plotly**

In [16]:
CRW_2d_df_6 = generate_crw(steps=1000, speed=6, coef=0.6)
save_df_to_csv(CRW_2d_df_6, 'trajectories_', 'CRW_6')

CRW_2d_df_9 = generate_crw(steps=1000, speed=9, coef=0.9)
save_df_to_csv(CRW_2d_df_9, 'trajectories_', 'CRW_9')

In [17]:
# aux to store turning angles
aux_ta_CRW_2d_df_6 = np.array([
    turning_angle(CRW_2d_df_6.iloc[i-1],
                    CRW_2d_df_6.iloc[i],
                    CRW_2d_df_6.iloc[i+1])
    for i in range(1, CRW_2d_df_6.shape[0] - 1)
])

# aux to store turning angles
aux_ta_CRW_2d_df_9 = np.array([
    turning_angle(CRW_2d_df_9.iloc[i-1],
                    CRW_2d_df_9.iloc[i],
                    CRW_2d_df_9.iloc[i+1])
    for i in range(1, CRW_2d_df_9.shape[0] - 1)
])
    
# Save to pandas DF
df_ta_CRW6_CRW9 = np_arrays_to_df(arrays=[aux_ta_CRW_2d_df_6, aux_ta_CRW_2d_df_9], column_names=['ta_CRW_c6', 'ta_CRW_c9'])

# Write to csv
save_df_to_csv(df_in=df_ta_CRW6_CRW9, path='./metrics', name="met_ta_crw_c6_crw_c9")


In [18]:
# Check documentation
# https://plotly.com/python/histograms/

resolution = 360
# PLot histogram
fig_met_df_3 = go.Figure()


# Histogram turning angle CRW_2d_df_6
## start - Add your code here
fig_met_df_3.add_trace(
    go.Histogram(x=df_ta_CRW6_CRW9.ta_CRW_c6,
    name='CRW observed c=0.6',
    histnorm='percent',
    xbins=dict( # bins used for histogram
        start=-np.pi,
        end=np.pi,
        size=np.pi/resolution
    ),
    marker_color='#3333FF',
    opacity=0.5)
)
## end - Add your code here


# Histogram turning angle CRW_2d_df_9
## start - Add your code here
fig_met_df_3.add_trace(
    go.Histogram(x=df_ta_CRW6_CRW9.ta_CRW_c9,
    histnorm='percent',
    name='CRW observed c=0.9', 
    xbins=dict( # bins used for histogram
        start=-np.pi,
        end=np.pi,
        size=np.pi/resolution
    ),
    marker_color='#FF3333',
    opacity=0.5)
)
## end - Add your code here

domain = np.linspace(-np.pi, np.pi, resolution)
wc6_t = np.array([wrapcauchy.pdf(i, c=0.6) for i in np.linspace(0, 2*np.pi, resolution)])
wc9_t = np.array([wrapcauchy.pdf(i, c=0.9) for i in np.linspace(0, 2*np.pi, resolution)])
wc6 = np.concatenate((wc6_t[int(resolution/2):resolution], wc6_t[0:int(resolution/2)]), axis=0)
wc9 = np.concatenate((wc9_t[int(resolution/2):resolution], wc9_t[0:int(resolution/2)]), axis=0)
fig_met_df_3.add_traces(
    go.Scatter(
        x = domain,
        y = wc6,
        name = 'Cauchy expected c=0.6',
        mode = 'lines',
        marker_color='#3333FF'
    )
)   

fig_met_df_3.add_traces(
    go.Scatter(
        x = domain,
        y = wc9,
        name = 'Cauchy expected c=0.9',
        mode = 'lines',
        marker_color='#FF3333'
    )
)   

fig_met_df_3.update_layout(barmode='group')
fig_met_df_3.show()

## Actividad 4: Step-length Distribution - (Dist. origen vs Dist. observada)

* Generar Levy Flight
* Guardar trayectorias en **Pandas** Data Frame
* Guardar metricas en **numpy** array
* Obtener Step-length distribution
* Comparar en gráfica distribución origen vs distribución observada
* Visualizar con **plotly**

In [19]:
# Create and save trajectories
lf_steps = 10000
lf_speed = 6
lf_cauchy = .7
lf_beta = 1
lf_std_steps = 3
Levy_2d_df_1 = generate_levy(steps=lf_steps, speed=lf_speed, cauchy=lf_cauchy, alpha=1.1, beta=lf_beta, std_steps=lf_std_steps)
save_df_to_csv(Levy_2d_df_1, './trajectories_', 'levy_6_1')

Levy_2d_df_7 = generate_levy(steps=lf_steps, speed=lf_speed, cauchy=lf_cauchy, alpha=1.7, beta=lf_beta, std_steps=lf_std_steps)
save_df_to_csv(Levy_2d_df_7, './trajectories_', 'levy_6_7')

# Load existing trajectories
#Levy_2d_df_1 = pd.read_csv('trajectories/levy_6_1.csv')

# Load existing trajectories
#Levy_2d_df_7 = pd.read_csv('trajectories/levy_6_7.csv')

In [20]:
# Show trajectories
# Init figure
fig_3d = go.Figure()

# Plot trajectory in 3-D space
fig_3d.add_trace(
    go.Scatter(x = Levy_2d_df_1.x,
                 y = Levy_2d_df_1.y,
                 marker = dict(size=2),
                 line = dict(color='blue', width=2),
                 mode = 'lines',
                 name = 'Levy 1',
                 showlegend = True))


fig_3d.add_trace(
    go.Scatter(x = Levy_2d_df_7.x,
                 y = Levy_2d_df_7.y,
                 marker = dict(size=2),
                 line = dict(color='red', width=2),
                 mode = 'lines',
                 name = 'Levy 7',
                 showlegend = True))

fig_3d.show()

In [21]:
# aux to store turning angles
aux_ta_Levy_2d_df_1 = np.array([
    abs(turning_angle(Levy_2d_df_1.iloc[i-1],
                    Levy_2d_df_1.iloc[i],
                    Levy_2d_df_1.iloc[i+1]))
    for i in range(1, Levy_2d_df_1.shape[0] - 1)
]
)
# aux to store step-lengths
aux_sl_Levy_2d_df_1 = np.empty(shape=(0))
i = 0
step_count = 1
while i < len(aux_ta_Levy_2d_df_1):
    if aux_ta_Levy_2d_df_1[i] == 0 and i < len(aux_ta_Levy_2d_df_1) - 1:
        step_count = step_count + 1
    else:
        aux_sl_Levy_2d_df_1 =  np.append(aux_sl_Levy_2d_df_1, step_count)
        step_count = 1
    i = i + 1


# aux to store turning angles
aux_ta_Levy_2d_df_7 = np.array([
    turning_angle(Levy_2d_df_7.iloc[i-1],
                    Levy_2d_df_7.iloc[i],
                    Levy_2d_df_7.iloc[i+1])
    for i in range(1, Levy_2d_df_7.shape[0] - 1)
])
# aux to store step-lengths
aux_sl_Levy_2d_df_7 = np.empty(shape=(0))
i = 0
step_count = 1
while i < len(aux_ta_Levy_2d_df_7):
    if aux_ta_Levy_2d_df_7[i] == 0 and i < len(aux_ta_Levy_2d_df_7) - 1:
        step_count = step_count + 1
    else:
        aux_sl_Levy_2d_df_7 =  np.append(aux_sl_Levy_2d_df_7, step_count)
        step_count = 1
    i = i + 1

In [22]:
# PLot histogram
fig_met_df_4 = go.Figure()

# Histogram step-length Levy_2d_df_1
## start - Add your code here
fig_met_df_4.add_trace(
    go.Histogram(x=aux_sl_Levy_2d_df_1[:],
    name='Levy 1',
    histnorm='probability',
    xbins=dict( # bins used for histogram
        size=1
    ),
    marker_color='#3333FF',
    opacity=0.5)
)

## end - Add your code here
# Histogram step-length Levy_2d_df_7
## start - Add your code here
fig_met_df_4.add_trace(
    go.Histogram(x=aux_sl_Levy_2d_df_7[:],
    name='Levy 7',
    histnorm='probability',
    xbins=dict( # bins used for histogram
       size=1
    ),
    marker_color='#FF3333',
    opacity=0.5)
)    
## end - Add your code here

# Add origin distributions
## start - Add your code here
maxi = math.ceil(max([max(aux_sl_Levy_2d_df_7), max(aux_sl_Levy_2d_df_1)]))
print(maxi)
lf_domain = np.linspace(0, math.ceil(maxi/8), maxi)
colors = np.array(['#3333FF', '#FF3333'])
col = 0
for exp in [1.7, 1.1]:
    fig_met_df_4.add_traces(
        go.Scatter(
            x = lf_domain,
            y = np.array([levy_stable.pdf(x=i, alpha=exp, beta=lf_beta,loc=lf_std_steps) for i in lf_domain]),
            name = f'Levy Dist α={exp} β={lf_beta}',
            mode = 'lines',
            marker_color=colors[col]
        )
    )
    col += 1  
## end - Add your code here


fig_met_df_4.show()

254
